# 分類タスクgeneratorのサンプル

In [ ]:
import glob
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

class KarutaClsGenerator:
    def __init__(self):
        # 画像フォルダ
        self.imgdir = "dataset_cls"
        # データ
        with open("kimariji_abbrev.txt") as f:
            kimarijis = [l.strip() for l in f.readlines()]
        self.xs = []  # list(np.array)
        self.ys = []  # list(int)
        for label in range(15):
            kimariji: str = kimarijis[label]
            dirname = "{:03d}_{}".format(label + 1, kimariji)
            dir = os.path.join(self.imgdir, dirname)
            for path in glob.glob(os.path.join(dir, "*.png")):
                pil_img = Image.open(path)
                img = np.array(pil_img)
                self.xs.append(img)
                self.ys.append(label)

    def gen_series(self):
        for self.id in range(len(self)):
            yield self.xs[self.id], self.ys[self.id]

    def __len__(self):
        return len(self.ys)

cls_gen = KarutaClsGenerator()

In [ ]:
from matplotlib import pyplot as plt
import tensorflow as tf

ds = tf.data.Dataset.from_generator(
    cls_gen.gen_series,
    output_types=(tf.uint8, tf.uint8),
    output_shapes=((None, None, 3), ()))

batch_size = 2
ds_batch = ds.shuffle(20).padded_batch(batch_size)
xs, ys = next(iter(ds_batch))
for x, y in zip(xs, ys):
    fig, ax = plt.subplots()
    ax.imshow(x)
    print(y.numpy())

In [ ]:
import tensorflow as tf

img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)  # [0, 1], float32
images, labels = next(img_gen.flow_from_directory("dataset_cls", target_size=(224, 224)))
print(images.dtype, images.shape)
print(labels.dtype, labels.shape)

In [ ]:
for images, labels in ds.take(1):
    print("images.shape: ", images.shape)
    print("labels.shape: ", labels.shape)

In [ ]:
from matplotlib import pyplot as plt
for images, labels in ds.take(1):
    for image, label in zip(images, labels):
        fig, ax = plt.subplots()
        ax.imshow(image)
        #print(label.numpy())